# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 7 2022 11:01AM,241294,20,8243540_ASN,"ACI Healthcare USA, Inc.",Released
1,Jul 7 2022 10:40AM,241291,19,8243517,"Emersa Waterbox, LLC",Released
2,Jul 7 2022 10:28AM,241290,19,8243508,"Emersa Waterbox, LLC",Released
3,Jul 7 2022 10:25AM,241289,19,8243507,"Emersa Waterbox, LLC",Released
4,Jul 7 2022 10:24AM,241288,10,8239705,Eywa Pharma Inc.,Released
5,Jul 7 2022 10:24AM,241288,10,8239707,Eywa Pharma Inc.,Released
6,Jul 7 2022 10:24AM,241288,10,8239710,Eywa Pharma Inc.,Released
7,Jul 7 2022 10:24AM,241288,10,8239714,Eywa Pharma Inc.,Released
8,Jul 7 2022 10:24AM,241288,10,8239715,Eywa Pharma Inc.,Released
9,Jul 7 2022 10:24AM,241288,10,8239716,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
52,241288,Released,20
53,241289,Released,1
54,241290,Released,1
55,241291,Released,1
56,241294,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
241288,NaN,20.0
241289,NaN,1.0
241290,NaN,1.0
241291,NaN,1.0
241294,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
241201,1.0,12.0
241203,0.0,9.0
241204,4.0,0.0
241208,0.0,1.0
241213,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
241201,1,12
241203,0,9
241204,4,0
241208,0,1
241213,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,241201,1,12
1,241203,0,9
2,241204,4,0
3,241208,0,1
4,241213,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,241201,1,12
1,241203,,9
2,241204,4,
3,241208,,1
4,241213,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 7 2022 11:01AM,241294,20,"ACI Healthcare USA, Inc."
1,Jul 7 2022 10:40AM,241291,19,"Emersa Waterbox, LLC"
2,Jul 7 2022 10:28AM,241290,19,"Emersa Waterbox, LLC"
3,Jul 7 2022 10:25AM,241289,19,"Emersa Waterbox, LLC"
4,Jul 7 2022 10:24AM,241288,10,Eywa Pharma Inc.
24,Jul 7 2022 10:18AM,241286,10,"Senseonics, Incorporated"
25,Jul 7 2022 10:17AM,241285,15,"Laser Pharmaceuticals, LLC"
37,Jul 7 2022 10:11AM,241282,15,"Alliance Pharma, Inc."
55,Jul 7 2022 10:11AM,241284,20,"Exact-Rx, Inc."
57,Jul 7 2022 10:09AM,241283,10,MedStone Pharma LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jul 7 2022 11:01AM,241294,20,"ACI Healthcare USA, Inc.",,1
1,Jul 7 2022 10:40AM,241291,19,"Emersa Waterbox, LLC",,1
2,Jul 7 2022 10:28AM,241290,19,"Emersa Waterbox, LLC",,1
3,Jul 7 2022 10:25AM,241289,19,"Emersa Waterbox, LLC",,1
4,Jul 7 2022 10:24AM,241288,10,Eywa Pharma Inc.,,20
5,Jul 7 2022 10:18AM,241286,10,"Senseonics, Incorporated",,1
6,Jul 7 2022 10:17AM,241285,15,"Laser Pharmaceuticals, LLC",6,6
7,Jul 7 2022 10:11AM,241282,15,"Alliance Pharma, Inc.",,18
8,Jul 7 2022 10:11AM,241284,20,"Exact-Rx, Inc.",,2
9,Jul 7 2022 10:09AM,241283,10,MedStone Pharma LLC,,12


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 7 2022 11:01AM,241294,20,"ACI Healthcare USA, Inc.",1,
1,Jul 7 2022 10:40AM,241291,19,"Emersa Waterbox, LLC",1,
2,Jul 7 2022 10:28AM,241290,19,"Emersa Waterbox, LLC",1,
3,Jul 7 2022 10:25AM,241289,19,"Emersa Waterbox, LLC",1,
4,Jul 7 2022 10:24AM,241288,10,Eywa Pharma Inc.,20,
5,Jul 7 2022 10:18AM,241286,10,"Senseonics, Incorporated",1,
6,Jul 7 2022 10:17AM,241285,15,"Laser Pharmaceuticals, LLC",6,6
7,Jul 7 2022 10:11AM,241282,15,"Alliance Pharma, Inc.",18,
8,Jul 7 2022 10:11AM,241284,20,"Exact-Rx, Inc.",2,
9,Jul 7 2022 10:09AM,241283,10,MedStone Pharma LLC,12,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 7 2022 11:01AM,241294,20,"ACI Healthcare USA, Inc.",1,
1,Jul 7 2022 10:40AM,241291,19,"Emersa Waterbox, LLC",1,
2,Jul 7 2022 10:28AM,241290,19,"Emersa Waterbox, LLC",1,
3,Jul 7 2022 10:25AM,241289,19,"Emersa Waterbox, LLC",1,
4,Jul 7 2022 10:24AM,241288,10,Eywa Pharma Inc.,20,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jul 7 2022 11:01AM,241294,20,"ACI Healthcare USA, Inc.",1.0,NaN
1,Jul 7 2022 10:40AM,241291,19,"Emersa Waterbox, LLC",1.0,NaN
2,Jul 7 2022 10:28AM,241290,19,"Emersa Waterbox, LLC",1.0,NaN
3,Jul 7 2022 10:25AM,241289,19,"Emersa Waterbox, LLC",1.0,NaN
4,Jul 7 2022 10:24AM,241288,10,Eywa Pharma Inc.,20.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 7 2022 11:01AM,241294,20,"ACI Healthcare USA, Inc.",1.0,0.0
1,Jul 7 2022 10:40AM,241291,19,"Emersa Waterbox, LLC",1.0,0.0
2,Jul 7 2022 10:28AM,241290,19,"Emersa Waterbox, LLC",1.0,0.0
3,Jul 7 2022 10:25AM,241289,19,"Emersa Waterbox, LLC",1.0,0.0
4,Jul 7 2022 10:24AM,241288,10,Eywa Pharma Inc.,20.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,5790241,305.0,4.0
12,723757,9.0,9.0
15,965076,81.0,10.0
16,723709,1.0,2.0
19,1206352,64.0,2.0
20,1206276,46.0,9.0
21,1688737,7.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,5790241,305.0,4.0
1,12,723757,9.0,9.0
2,15,965076,81.0,10.0
3,16,723709,1.0,2.0
4,19,1206352,64.0,2.0
5,20,1206276,46.0,9.0
6,21,1688737,7.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,305.0,4.0
1,12,9.0,9.0
2,15,81.0,10.0
3,16,1.0,2.0
4,19,64.0,2.0
5,20,46.0,9.0
6,21,7.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,305.0
1,12,Released,9.0
2,15,Released,81.0
3,16,Released,1.0
4,19,Released,64.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Executing,4.0,9.0,10.0,2.0,2.0,9.0,0.0
Released,305.0,9.0,81.0,1.0,64.0,46.0,7.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Executing,4.0,9.0,10.0,2.0,2.0,9.0,0.0
1,Released,305.0,9.0,81.0,1.0,64.0,46.0,7.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Executing,4.0,9.0,10.0,2.0,2.0,9.0,0.0
1,Released,305.0,9.0,81.0,1.0,64.0,46.0,7.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()